In [1]:
import torch
from torch.utils.data.dataloader import DataLoader

import sys

# Setting path so as we can find files in ../src folder
sys.path.append('../src')

from BRATS2013Dataset import BRATS2013Dataset

from tqdm.autonotebook import tqdm

import numpy as np

import json

import skimage

from pprint import pprint

from matplotlib import pyplot as plt

import modin.pandas as pd

import os
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Dask

from modin.config import ProgressBar
ProgressBar.enable()

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import pytz

In [2]:
DATASET_NAME = "brats_2013"
PATCH_SIZE = 65
DATASET_SPLIT = "train"
IS_BALANCED = "balanced"

In [3]:
DF_PATH = f"../data/{DATASET_NAME}_patch_{PATCH_SIZE}_df_{DATASET_SPLIT}.json"

In [4]:
df = pd.read_json(DF_PATH)

2023-01-02 17:24:08,578	INFO worker.py:1538 -- Started a local Ray instance.


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [5]:
DATASET_SIZE = 50000

TRAIN_PERCENTAGE = 0.7
VAL_PERCENTAGE = 0.2

DATASET_SIZE_TRAIN = int(DATASET_SIZE * TRAIN_PERCENTAGE)
DATASET_SIZE_VAL = int(DATASET_SIZE * VAL_PERCENTAGE)
DATASET_SIZE_TEST = DATASET_SIZE - DATASET_SIZE_TRAIN - DATASET_SIZE_VAL

In [6]:
DATASET_SIZE_TRAIN, DATASET_SIZE_VAL, DATASET_SIZE_TEST

(35000, 10000, 5000)

In [7]:
NUM_LABELS = 6

In [8]:
DATASET_SIZE_TRAIN_PER_LABEL = DATASET_SIZE_TRAIN // NUM_LABELS
DATASET_SIZE_VAL_PER_LABEL = DATASET_SIZE_VAL // NUM_LABELS
DATASET_SIZE_TEST_PER_LABEL = DATASET_SIZE_TEST // NUM_LABELS

In [9]:
DATASET_SIZE_TRAIN_PER_LABEL, DATASET_SIZE_VAL_PER_LABEL, DATASET_SIZE_TEST_PER_LABEL

(5833, 1666, 833)

In [10]:
dataset_size_split_per_label = {
  "train": DATASET_SIZE_TRAIN_PER_LABEL,
  "val": DATASET_SIZE_VAL_PER_LABEL,
  "test": DATASET_SIZE_TEST_PER_LABEL,
}

In [11]:
labels = range(NUM_LABELS)

subsampling_indexes_dict = {}

In [12]:
for label in tqdm(labels):
  
  print("\n\n")

  df_filtered = df[df['label'] == label] 

  # print(f"Labels in the filtered df: {df_filtered['label'].unique()}")

  subsampling_indexes = np.random.choice(
    df_filtered.index, 
    size=dataset_size_split_per_label[DATASET_SPLIT], replace=False
  ) 

  subsampling_indexes_dict[str(label)] = subsampling_indexes

  0%|          | 0/6 [00:00<?, ?it/s]

Estimated completion of line 17:   0%           Elapsed time: 00:00, estimated remaining time: ?

(raylet) Error processing line 1 of /home/dansolombrino/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/dansolombrino/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(raylet)   pytz: No module named 'pytz'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of /home/dansolombrino/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/dansolombrino/.local/li

In [13]:
# subsampling_indexes_dict

In [14]:
subsampling_indexes_all_labels = np.asarray(list(subsampling_indexes_dict.values()))

In [15]:
subsampling_indexes_all_labels = subsampling_indexes_all_labels.reshape(-1)

In [16]:
np.unique(subsampling_indexes_all_labels).shape == subsampling_indexes_all_labels.shape

True

In [17]:
df_to_export = df.iloc[subsampling_indexes_all_labels]

In [18]:
DATASET_SUBSAMPLE_ID = 0

dataset_name = DATASET_NAME.replace("_", "").upper()
DATASET_SUBSAMPLE_PATH = f"../data/{dataset_name}_patches_{PATCH_SIZE}_{IS_BALANCED}/{DATASET_SUBSAMPLE_ID}"

if not os.path.exists(DATASET_SUBSAMPLE_PATH):
  os.makedirs(DATASET_SUBSAMPLE_PATH)

Estimated completion of line 17:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [19]:
df_to_export.to_json(f"{DATASET_SUBSAMPLE_PATH}/{DATASET_SPLIT}_df.json")

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?